In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Time-variant Properties (e.g. coupon schedule) in LUSID 

Attributes
----------
coupon schedules
multi-valued properties
time-variant properties
"""

toggle_code("Toggle Docstring")

## Time-variant Properties

This notebook illustrates the use of time-variant properties, which are a type of property that depend on different effective dates. 

In the example below we use a quarterly ratings schedule as a demonstrative example, showing how the LUSID API can be used to query values on different effective dates. We will also demonstrate the bi-temporality of the data, using different as at dates.  

In [2]:
# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python

import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import libraries
from datetime import datetime, timedelta
import time
import pytz
import json
import os
import pandas as pd

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Configure notebook logging and warnings
import logging
logging.basicConfig(level=logging.INFO)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

# Import required LUSID APIs
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)

## 1. Load Data

### 1.1 Instruments

Load the instruments data into LUSID, and set the  to which we can later add the desired properties. The dataset in the example containins large cap stocks from the FTSE 100 Index. 

In [3]:
scope = "TimeVariant"

df = pd.read_csv("data/equity_transactions_isin.csv").drop_duplicates()
df.head()

,portfolio_code,portfolio_name,portfolio_base_currency,ISIN,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,216238,equity,EQ_1234,Aviva,txn-1,StockIn,02/01/2020,04/01/2020,120000,4.23,600000,GBP,ftse_tracker,nan
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB00BH0P3Z91,BH0P3Z9,equity,EQ_1235,BHP,txn-2,StockIn,02/01/2020,04/01/2020,60000,17.89,1080000,GBP,ftse_tracker,nan
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,3134865,equity,EQ_1236,Barclays,txn-3,StockIn,02/01/2020,04/01/2020,150000,1.80,300000,GBP,ftse_tracker,nan
3,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,798059,equity,EQ_1237,BP,txn-4,StockIn,02/01/2020,04/01/2020,100000,4.75,500000,GBP,ftse_tracker,nan
4,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,540528,equity,EQ_1238,HSBC,txn-5,StockIn,02/01/2020,04/01/2020,20000,5.89,120000,GBP,ftse_tracker,nan


In [4]:
# Create a mapping schema for the instruments in the portfolio
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "ISIN"
    },
    "required": {
        "name": "name"
    },
}


# Instruments can be loaded using a dataframe with file_type set to "instruments"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,10,0,0


### 1.2 Create the time-variant properties

With the instruments in LUSID, we can now define the properties and add them to the selected instruments. To this end we will need to define a _property defintion_, where we also specify the <lifetime> of the property as _'TimeVariant'_.  

In [5]:
# Setup the property details
property_name = "QuarterlyRating"

def create_property(property_name):
    # Create the property definition request
    global property_definition
    property_definition = models.CreatePropertyDefinitionRequest(
                domain="Instrument",
                scope=scope,
                code=f"TimeVariant-{property_name}",
                display_name=property_name,
                data_type_id=lusid.ResourceId(scope="system", code="number"),
                life_time="TimeVariant",
            )

    # create property definition
    try:
        property_definition_request = property_definitions_api.create_property_definition(
            create_property_definition_request=property_definition
        )

    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            logging.info(
                f"Property {property_definition.domain}/{property_definition.scope}/{property_definition.code} already exists"
            )


create_property(property_name)

INFO:root:Property Instrument/TimeVariant/TimeVariant-QuarterlyRating already exists


### 1.3 Upsert the instrument properties

In order to upsert the instrument property, we create a function defining the body of our property request and use the instruments_api to pass the values for our instrument. In this case we've used Aviva as an example, to which we are passing dates and numeric values to the property's effective date and value. 

In [6]:
# create a function to upsert the properties for a selected instrument using instruments_api
def upsert_instrument_property(ISIN, value, effectiveFrom):
    property_request = [
        models.UpsertInstrumentPropertyRequest(
            identifier_type="ClientInternal",
            identifier=ISIN,
            properties=[
                models.ModelProperty(
                    key=f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}",
                    value=models.PropertyValue(
                        metric_value=models.MetricValue(
                            value=value
                        )
                    ),
                    effective_from=effectiveFrom
                )
            ]
        )
    ]

    instruments_api.upsert_instruments_properties(
                        upsert_instrument_property_request=property_request)

# pass in the schedule effective dates and values for a selected ISIN
instrument_id = "GB0002162385"
schedule = [
    { "2020-12-31" : "5"},
    { "2021-03-31" : "4"},
    { "2021-06-30" : "3"},
    { "2021-09-30" : "3"},
]


for element in schedule:
    for key in element:
        date = datetime.strptime(key, "%Y-%m-%d").astimezone(pytz.utc)
        upsert_instrument_property(instrument_id, float(element[key]), effectiveFrom=date)
        print(f"Upserting ISIN: {instrument_id}, Value: {float(element[key])}, Eff. Date: {date}")

ISIN: GB0002162385, Value: 5.0, Eff. Date: 2020-12-31 00:00:00+00:00
ISIN: GB0002162385, Value: 4.0, Eff. Date: 2021-03-30 23:00:00+00:00
ISIN: GB0002162385, Value: 3.0, Eff. Date: 2021-06-29 23:00:00+00:00
ISIN: GB0002162385, Value: 3.0, Eff. Date: 2021-09-29 23:00:00+00:00


## 2. Querying Instrument Properties

### 2.1 Get properties by effective dates

We can use [_get_instruments_](https://www.lusid.com/docs/api#operation/GetInstruments) call to the API along with the instrument property key in order to query the previously upserted data. 

In [7]:
# Create a function to query properties for a given instrument_id and effective date
def get_properties(ISIN, effective_date):
    response = instruments_api.get_instruments(
        identifier_type="ClientInternal",
        request_body=[ISIN],
        property_keys=[f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"],
        effective_at=datetime.strptime(effective_date, "%Y-%m-%d").astimezone(pytz.utc),
    )
    # return the effective date and property value
    return (response.values[ISIN].properties[0].effective_from,
            response.values[ISIN].properties[0].value.metric_value.value)

data = [get_properties(instrument_id, "2021-06-30")]
df = pd.DataFrame(data, columns=['Effective Date', 'Value'])
df

,Effective Date,Value
0,2021-06-29 23:00:00+00:00,3.00


### 2.2 Viewing properties by different  _'as at'_ dates

The below is an example of the bi-temporality of data stored in LUSID, which means that we can view data as it was on a certain 'as at' date. In this case, we will assume that there was an update to the entries with our previous ratings example, requiring an amendment to the property values. 

Given the bi-temporal feature, we will still be able to view the original entries as they were before this amendment was made. In this example, we update the expected ratings using a hypothetical downgrade.  

In [8]:
# Upsert the new set of data for the same instrument used before

new_schedule = [
    { "2020-12-31" : "5"},
    { "2021-03-31" : "2"},
    { "2021-06-30" : "1"},
    { "2021-09-30" : "1"},
]

# In order to upsert the new values at a different asAt time, we can delay the upsert of the new values
time.sleep(30)

for element in new_schedule:
    for key in element:
        date = datetime.strptime(key, "%Y-%m-%d").astimezone(pytz.utc)
        upsert_instrument_property(instrument_id, float(element[key]), effectiveFrom=date)
        print(f"Upserting ISIN: {instrument_id}, Value: {float(element[key])}, Eff. Date: {date}")

ISIN: GB0002162385, Value: 5.0, Eff. Date: 2020-12-31 00:00:00+00:00
ISIN: GB0002162385, Value: 2.0, Eff. Date: 2021-03-30 23:00:00+00:00
ISIN: GB0002162385, Value: 1.0, Eff. Date: 2021-06-29 23:00:00+00:00
ISIN: GB0002162385, Value: 1.0, Eff. Date: 2021-09-29 23:00:00+00:00


The _get_instruments_ call from the instruments API can be queried with the additional _'as at'_ parameter, which we will add to the get properties function below.  

In [9]:
# Create a function to query properties for a given instrument_id and effective date
def get_properties_as_at(ISIN, effective_date, as_at_date):
    response = instruments_api.get_instruments(
        identifier_type="ClientInternal",
        request_body=[ISIN],
        property_keys=[f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"],
        effective_at=datetime.strptime(effective_date, "%Y-%m-%d").astimezone(pytz.utc),
        as_at = as_at_date
    )
    # return the effective date and property value
    return (response.values[ISIN].properties[0].effective_from,
            as_at_date,
            response.values[ISIN].properties[0].value.metric_value.value)

time_now = datetime.now().astimezone(pytz.utc) 

In [10]:
# Set the as_at value to before the change was made
as_at_before = time_now - timedelta(seconds=30)
data_old = [get_properties_as_at(instrument_id, "2021-06-30", as_at_before)]
df_old = pd.DataFrame(data_old, columns=['Effective Date', 'As at Date','Value'])
display(df_old)


,Effective Date,As at Date,Value
0,2021-06-29 23:00:00+00:00,2020-10-23 07:30:32.813434+00:00,3.00


In [11]:
# Set the as_at value to illustrate the updated value for the same effective dt
data_new = [get_properties_as_at(instrument_id, "2021-06-30", time_now)]
df_new = pd.DataFrame(data_new, columns=['Effective Date', 'As at Date','Value'])
display(df_new)

,Effective Date,As at Date,Value
0,2021-06-29 23:00:00+00:00,2020-10-23 07:31:02.813434+00:00,1.00
